# Building a vectorizer model with gensim

In [3]:
import gensim
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
df=pd.read_csv("../data/translated/kurdish-reviews9.csv")
scc=pd.read_csv('../scrapedData/xendan-7000.csv')
# df=pd.read_csv("../data/medical-kurdish-dataset.csv")

In [5]:
df.shape    

(75279, 11)

In [6]:
scc.shape

(7000, 5)

In [7]:
df['KurdishText']

0        چەندین بەرهەمی خۆراکی سەگی قوتووکراوی ڤیتالیتی...
1        بەرهەمەکە گەیشتە لای کە بە ناوی Jumbo Salted P...
2        ئەمە شیرینییەکە کە لە دەوروبەری چەند سەدەیەکدا...
3        ئەگەر بەدوای پێکهاتەی نهێنی ڕۆبیتوسیندا دەگەڕێ...
4        تافی نایاب بە نرخێکی نایاب. جۆرێکی بەرفراوانی ...
                               ...                        
75274    ئێستا نزیکەی ٥ مانگە ئەم خواردنە پشیلەیە بەکار...
75275    دوای لێکۆڵینەوە لە هەڵسەنگاندنی خۆراکی ئاژەڵە ...
75276    ئەمە خواردنێکی پشیلە وشکی کوالێتی بەرزە کە هەس...
75277    پشیلەکەم نزیکەی ساڵێکە ئەم براندە دەخوات و هیچ...
75278    پێم وابوو یارمەتیدەر دەبێت کە بەراورد بکەم بە ...
Name: KurdishText, Length: 75279, dtype: object

In [8]:
scc['Article']

0       خەندان-\nدوای شه‌ش ڕۆژ له‌ ده‌ستپێكردنی هێرشه‌...
1       خەندان-\nكرێكارێكی خه‌ڵكی رۆژهه‌ڵاتی كوردستان ...
2       خەندان-\nبه‌رپرسانی وه‌زاره‌تی به‌رگری ئه‌مریك...
3       خه‌ندان-\n\nنرخی نەوتی خاو لە سەرەتای مامەڵەکا...
4       خەندان-\n\nوه‌زاره‌تی كاره‌بای عێراق ئه‌مڕۆ پێ...
                              ...                        
6995    خەندان-\nكەوتنی جۆو بایدن سەرۆكی ویلایەتە یەكگ...
6996    خەندان-\nئەحمەد خەڵەف سەرۆكی دەستەی چاودێری كۆ...
6997                                                  NaN
6998    خەندان-\nسەرچاوەیەکی ئەمنی لە پارێزگای هەولێر ...
6999    خەندان-\n\nپارێزەر و شارەزایەکی بواری یاسایی ر...
Name: Article, Length: 7000, dtype: object

In [9]:
scc['Article'] = scc['Article'].fillna('')

In [6]:
df['KurdishText'] = df['KurdishText'].fillna('')

In [7]:

# res={}
# for i,row in tqdm(df.iterrows(),total=len(df)):
#     try:
#         text=row.KurdishText
#         myId=row.Id
#         gensim_results=gensim.utils.simple_preprocess(text)
#         res[myId]=gensim_results
#     except TypeError:
#         print("typerror for text: ",text)
#         continue



### tokenizing the kurdih reviews

In [8]:
# continuing with old way not looping
review_text=df.KurdishText.apply(gensim.utils.simple_preprocess)

In [10]:
# continuing with old way not looping
review_text=scc.Article.apply(gensim.utils.simple_preprocess)

In [11]:
review_text

0       [خەندان, دوای, شه, ڕۆژ, له, ده, ستپێكردنی, هێر...
1       [خەندان, كرێكارێكی, خه, ڵكی, رۆژهه, ڵاتی, كورد...
2       [خەندان, به, رپرسانی, وه, زاره, تی, به, رگری, ...
3       [خه, ندان, نرخی, نەوتی, خاو, لە, سەرەتای, مامە...
4       [خەندان, وه, زاره, تی, كاره, بای, عێراق, ئه, م...
                              ...                        
6995    [خەندان, كەوتنی, جۆو, بایدن, سەرۆكی, ویلایەتە,...
6996    [خەندان, ئەحمەد, خەڵەف, سەرۆكی, دەستەی, چاودێر...
6997                                                   []
6998    [خەندان, سەرچاوەیەکی, ئەمنی, لە, پارێزگای, هەو...
6999    [خەندان, پارێزەر, شارەزایەکی, بواری, یاسایی, ر...
Name: Article, Length: 7000, dtype: object

### building the model and training word2vec

In [12]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [13]:
model.build_vocab(review_text,progress_per=100)

In [14]:
model.epochs

5

In [15]:
model.corpus_count

7000

In [16]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(6132392, 7147150)

In [17]:
model.save(f"../models/xendan7000.model")

In [28]:
model.wv.most_similar("بێهۆش")

[('دایکی', 0.9240180850028992),
 ('پیاوێک', 0.92145836353302),
 ('هاوڕێکانی', 0.9199870824813843),
 ('ژنەکەی', 0.9163362979888916),
 ('روداوەکانیش', 0.9147975444793701),
 ('لێکەوتوەتەوە', 0.9103312492370605),
 ('لێدانی', 0.908986508846283),
 ('گەیەنرایە', 0.908150315284729),
 ('ئۆتۆمبێلەکەی', 0.9069851040840149),
 ('ساچمە', 0.9066586494445801)]

In [19]:
model.wv.similarity(w1="دڵتەنگ",w2="دڵخۆش")

0.66419977

In [20]:
model.wv.doesnt_match(["گونجاو","هەرزان","باش","پرتەقاڵ"])

'هەرزان'

In [36]:
model.wv.doesnt_match(["ئاو","چای","پشیلە","قاوە","شیر"])


'چای'

In [37]:
model.wv.doesnt_match(["شێر","سەگ","پشیلە","باڵندە","پرتەقاڵ"])


'شێر'

In [38]:
model.wv.doesnt_match(["دڵخۆش","گران","دڵتەنگ","خەمبار"])


'گران'

In [40]:
model.wv.most_similar(positive=["ژن","پاشا"],negative=["پیاو"])

[('ریزەکانی', 0.8832405805587769),
 ('بینیویەتی', 0.8755570650100708),
 ('بۆردومانەکە', 0.873737096786499),
 ('شنگال', 0.8732637763023376),
 ('ئێندزێ', 0.870159924030304),
 ('تیرۆرستان', 0.8671587109565735),
 ('ئوردوگای', 0.8669210076332092),
 ('چەكدارانی', 0.8661324977874756),
 ('گارە', 0.8660470247268677),
 ('بانە', 0.8655515313148499)]

In [26]:
model.wv["باش"]

array([-0.46548828, -0.42102528, -0.12997295, -0.10357738, -0.9814635 ,
       -0.5446731 , -0.19553725,  0.7709324 , -0.07891028, -0.26040992,
       -1.0191065 , -0.25812966,  0.03735895,  0.10042942,  0.27474138,
       -0.02756481, -0.12971783,  0.2692504 , -0.32766986, -1.1883205 ,
        1.2441928 ,  0.04271733, -0.13483304,  0.06703393, -0.18480209,
       -0.5009575 ,  0.47007695, -0.8846807 ,  0.28579834,  0.01531253,
       -0.5319116 ,  0.84506506,  1.0272013 ,  0.41116464, -0.54708654,
        0.13905041,  0.14502011, -0.4891676 ,  0.03439448, -0.7306718 ,
        0.5518998 , -0.65784633,  0.9183559 , -0.06695156,  0.25491264,
       -0.27141535,  0.2662067 , -0.846691  ,  0.14537011,  0.4080283 ,
       -0.26782453,  0.0751069 , -0.4518882 ,  0.81795233, -0.18025246,
       -0.09285759, -0.12398448, -0.9052619 ,  0.26860058,  0.48979884,
        0.56360525,  0.08040442, -0.01943566, -0.31836256, -0.16249953,
        0.98453724, -0.17362545,  0.11234435,  0.06983974,  0.23

In [27]:
model.wv.distance(w1="ناوازە",w2="نایاب")

0.4809352159500122

## Comparison between 2000 model and 20,000 model

In [4]:
two_model=gensim.models.Word2Vec.load("../models/kurdish-word2vec-3000.model")
twenty_model=gensim.models.Word2Vec.load("../models/kurdish-word2vec-75279.model")
eng=gensim.models.Word2Vec.load("../models/english-word2vec.model")



#### Similiarity comparison

In [5]:
text="دڵتەنگ"
texteng="cat"
print(f'model 1: {two_model.wv.most_similar(text,topn=5)}')
print(f'model 2: {twenty_model.wv.most_similar(text,topn=5)}')
print(f'model 2: {eng.wv.most_similar(texteng,topn=5)}')

model 1: [('تارگێت', 0.9552736878395081), ('دەیفرۆشن', 0.9551293253898621), ('ئەوەندەی', 0.9550510048866272), ('فڕێدا', 0.9547895193099976), ('ستانداردی', 0.9546394348144531)]
model 2: [('بێهیوا', 0.9291748404502869), ('ناڕازی', 0.9205631017684937), ('کنجکاو', 0.9063196778297424), ('دەمارگیر', 0.8955897688865662), ('هیوادار', 0.8950649499893188)]
model 2: [('chewing', 0.7258464694023132), ('cats', 0.698319137096405), ('dog', 0.687409520149231), ('kid', 0.6554325222969055), ('toddler', 0.6033084392547607)]


In [38]:
text=["دۆلار","سەنت","شەربەت","پاوند"]

print(f'model 1: {two_model.wv.doesnt_match(text)}')
print(f'model 1: {twenty_model.wv.doesnt_match(text)}')

model 1: شەربەت
model 1: شەربەت


AttributeError: 'KeyedVectors' object has no attribute 'expandose_vectors'